In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

data = pd.read_csv('./HW3 附件/HW3_creditcard.csv')

In [91]:
def metrics(y_test, y_pred):
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.4f}")

    # Calculate precision
    precision = precision_score(y_test, y_pred)
    print(f"Precision: {precision:.4f}")

    # Calculate recall (sensitivity)
    recall = recall_score(y_test, y_pred)
    print(f"Recall: {recall:.4f}")

    # Calculate F1 score (harmonic mean of precision and recall)
    f1 = f1_score(y_test, y_pred)
    print(f"F1 Score: {f1:.4f}")
    # Calculate ROC AUC (Receiver Operating Characteristic Area Under the Curve)
    roc_auc = roc_auc_score(y_test, y_pred)
    print(f"ROC AUC: {roc_auc:.4f}\n")

In [92]:
df = data.copy()
target_column = 'Class'
test_size = 0.3
random_state = 42

# Seperate features and target
X = df.drop(columns=[target_column])
y = df[target_column]

# Split the data into training, validation, and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=random_state)

In [93]:
# Checking the data distribution
print("In all the data:")
print(f"Class 0: {(y == 0).sum()}")
print(f"Class 1: {(y == 1).sum()}")
print('')
print("In Train data:")
print(f"Class 0: {(y_train == 0).sum()}")
print(f"Class 1: {(y_train == 1).sum()}")
print('')
print("In Val data:")
print(f"Class 0: {(y_val == 0).sum()}")
print(f"Class 1: {(y_val == 1).sum()}")

In all the data:
Class 0: 28432
Class 1: 492

In Train data:
Class 0: 19909
Class 1: 337

In Val data:
Class 0: 8523
Class 1: 155


# Discussion Part

Class 0: 28432
Class 1: 492

In Train data:
Class 0: 19909
Class 1: 337

In Val data:
Class 0: 8523
Class 1: 155

With the data distribution of ['Class'], we can clearly see class 0 is more than 57 times of class 1.
The data is highly imbalance, we have to make use of recall or data augmented to validate the model performance.

In [94]:
print(X_train.shape)
print(X_val.shape)
print(X.shape)

(20246, 30)
(8678, 30)
(28924, 30)


In [95]:
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier(random_state=42)


def test_model(classifier):
    classifier.fit(X_train, y_train)
    # Predict on the training data and calculate accuracy
    y_pred = classifier.predict(X_train)
    accuracy = accuracy_score(y_train, y_pred)
    confusion = confusion_matrix(y_train, y_pred)
    print(confusion)
    metrics(y_train, y_pred)


    # Predict on the validation data and calculate accuracy
    y_pred = classifier.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    confusion = confusion_matrix(y_val, y_pred)
    print(confusion)
    metrics(y_val, y_pred)

test_model(decision_tree_classifier)


[[19909     0]
 [    0   337]]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
ROC AUC: 1.0000

[[8503   20]
 [  28  127]]
Accuracy: 0.9945
Precision: 0.8639
Recall: 0.8194
F1 Score: 0.8411
ROC AUC: 0.9085



In [96]:
weighted_tree_classifier = DecisionTreeClassifier(class_weight={0: 58, 1: 1}, random_state=42)
test_model(weighted_tree_classifier)


[[19909     0]
 [    0   337]]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
ROC AUC: 1.0000

[[8501   22]
 [  19  136]]
Accuracy: 0.9953
Precision: 0.8608
Recall: 0.8774
F1 Score: 0.8690
ROC AUC: 0.9374



In [97]:

# Resample data using SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

decision_tree_classifier = DecisionTreeClassifier(random_state=42)

# Fitting model on original data
decision_tree_classifier.fit(X_train, y_train)

# Fitting model on resampled data from SMOTE
decision_tree_classifier_resampled = DecisionTreeClassifier(random_state=42)
decision_tree_classifier_resampled.fit(X_train_resampled, y_train_resampled)

# Predict with original data
y_pred = decision_tree_classifier.predict(X_val)
print("Original:")
metrics(y_val, y_pred)

# Predict with resampled data
y_pred_resampled = decision_tree_classifier_resampled.predict(X_val)
print("After resampling based on SMOTE:")
metrics(y_val, y_pred_resampled)



Original:
Accuracy: 0.9945
Precision: 0.8639
Recall: 0.8194
F1 Score: 0.8411
ROC AUC: 0.9085

After resampling based on SMOTE:
Accuracy: 0.9892
Precision: 0.6533
Recall: 0.8387
F1 Score: 0.7345
ROC AUC: 0.9153



In [98]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_val)
print("XGBoost:")
metrics(y_val, y_pred)


XGBoost:
Accuracy: 0.9975
Precision: 0.9926
Recall: 0.8645
F1 Score: 0.9241
ROC AUC: 0.9322



In [99]:
xgb_model_r = xgb.XGBClassifier()
xgb_model_r.fit(X_train_resampled, y_train_resampled)

y_pred_resampled = xgb_model_r.predict(X_val)
print("After resampling based on SMOTE:")
metrics(y_val, y_pred_resampled)

After resampling based on SMOTE:
Accuracy: 0.9968
Precision: 0.9262
Recall: 0.8903
F1 Score: 0.9079
ROC AUC: 0.9445



In [100]:
xgb_model_ro = xgb.XGBClassifier()
xgb_model_ro.fit(X_train, y_train)
xgb_model_ro.fit(X_train_resampled, y_train_resampled)

y_pred_resampled = xgb_model_ro.predict(X_val)
print("After resampling based on SMOTE:")
metrics(y_val, y_pred_resampled)

After resampling based on SMOTE:
Accuracy: 0.9968
Precision: 0.9262
Recall: 0.8903
F1 Score: 0.9079
ROC AUC: 0.9445

